# **Economic Freedom and Happiness Across Europe**

In [1]:
import pandas as pd
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pycountry 

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Function for converting to iso 
def getIsoCode(country_name):
    try:
        country_iso = pycountry.countries.search_fuzzy(country_name)[0]
        return country_iso.alpha_3
    except LookupError:
        return ""

In [2]:
%%html
<style>
.cartesianlayer .bars path,
.colorbar .cbfills,
.slider-rail-rect { clip-path: inset(0% 0% 0% 0% round 6px); }

.legendundefined { clip-path: inset(0% 0% 0% 0% round 3px); }

.gtitle .line:nth-of-type(2) { fill: grey; }

.output_area:last-of-type,
.cell_output {
    filter: drop-shadow(0px 0px 10px rgba(0, 0, 0, 0.2));
    -webkit-backdrop-filter: blur(0); // fixes a safari drop-shadow bug
}

#notebook-container img,
main img {
    box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.2);
}

#notebook-container img,
main img,
.plotly-graph-div {
    border-radius: 10px;
    overflow: hidden;
    border: 2px solid black;
    border-bottom: 3px solid black;
}
</style>

In [3]:
happiness_data = pd.read_csv("happiness.csv")
happiness_data["Country"] = happiness_data["Country or region"]
happiness_data = happiness_data.drop(columns=["Country or region", "Overall rank"])
happiness_data = happiness_data.rename(columns={"Score": "Overall Happiness"})

In [4]:
freedom_data = pd.read_csv("freedom.csv", low_memory=False)
# Normalize country column
freedom_data = freedom_data.rename(columns={"countries": "Country"})

# Filter out years other than 2019
freedom_data["year"] = pd.to_numeric(freedom_data["year"], errors='coerce')
freedom_data = freedom_data[freedom_data["year"] == 2019]

# Reset and Drop
freedom_data = freedom_data.reset_index()
freedom_data = freedom_data.drop(columns="index")

freedom_filtered = freedom_data[freedom_data['Country'].isin([
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", 
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", 
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", 
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", 
    "Liechtenstein", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", 
    "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", 
    "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Vatican City"
])]

columns_to_keep = [
    'Country', 'ef_government', 'ef_gender', 'ef_legal', 'ef_money', 'ef_trade', 'ef_regulation', 'ef_rank'
]

freedom_filtered = freedom_filtered[columns_to_keep]

In [5]:
data = pd.merge(happiness_data, freedom_filtered, on='Country')
data = data.drop_duplicates(subset=['Country'])

data["iso"] = data['Country'].apply(getIsoCode)

# CREATING NEW RANK
sorted_df = data.sort_values(by='ef_rank')
sorted_df['ef_rank'] = range(1, len(sorted_df) + 1)

# Display the sorted DataFrame with ranks
data = sorted_df
data.head()

,Overall Happiness,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Country,ef_government,ef_gender,ef_legal,ef_money,ef_trade,ef_regulation,ef_rank,iso
5,7.480,1.452,1.526,1.052,0.572,0.263,0.343,Switzerland,7.717782,1.0,8.678721,9.874369,7.893539,8.269470,1,CHE
1,7.600,1.383,1.573,0.996,0.592,0.252,0.410,Denmark,5.343284,1.0,8.605796,9.901595,8.774736,8.539323,2,DNK
19,6.149,1.238,1.515,0.818,0.291,0.043,0.042,Lithuania,7.656798,1.0,7.181359,9.328248,8.680559,8.273870,3,LTU
10,7.021,1.499,1.553,0.999,0.516,0.298,0.310,Ireland,6.686180,1.0,7.680896,9.466828,8.730446,8.396136,4,IRL
9,7.054,1.333,1.538,0.996,0.450,0.348,0.278,United Kingdom,6.554639,1.0,7.692716,9.707107,8.433408,8.435807,5,GBR


## Analysis Across Countries

In [6]:
color_scale = px.colors.sequential.dense  # Example color scale

fig = px.choropleth(
    data,
    locations="iso",
    title='Economic Freedom Rank',
    color="ef_rank",
    hover_name="iso",
    hover_data=['iso'],
    color_continuous_scale=color_scale,
    projection="mercator",  # Ensure to use Mercator projection for global maps
)

fig.update_geos(
    projection_type="natural earth",  # Set the projection type
    center=dict(lon=15, lat=54),  # Set the center of the map to Europe
    scope="europe",  # Set the scope to Europe
    showcoastlines=True,  # Show coastlines for better geographic context
)

fig.update_geos(
    fitbounds="locations",  # Adjust the map bounds to fit the specified locations
)
# Set center to Europe
fig.update_geos(center=dict(lon=15, lat=54))

fig.show()

In [7]:
top_3 = data.nlargest(3, 'ef_rank')
bottom_3 = data.nsmallest(3, 'ef_rank')

In [8]:
# Convert data to pandas DataFrame
df = pd.DataFrame(data)

# Create Plotly scatter plot with dropdown menu
fig = go.Figure()

# Initial trace for Overall Happiness
fig.add_trace(go.Scatter(
    x=df["ef_rank"],
    y=df["Overall Happiness"],
    mode='lines+markers',  # Line plot with markers
    name='Overall Happiness'
))

# Define dropdown menu options
dropdown_options = [
    {'label': 'Overall Happiness', 'method': 'update', 'args': [{'y': [df["Overall Happiness"]]}]},
    {'label': 'GDP per capita', 'method': 'update', 'args': [{'y': [df["GDP per capita"]]}]},
    {'label': 'Social support', 'method': 'update', 'args': [{'y': [df["Social support"]]}]},
    {'label': 'Healthy life expectancy', 'method': 'update', 'args': [{'y': [df["Healthy life expectancy"]]}]},
    {'label': 'Freedom to make life choices', 'method': 'update', 'args': [{'y': [df["Freedom to make life choices"]]}]},
    {'label': 'Generosity', 'method': 'update', 'args': [{'y': [df["Generosity"]]}]},
    {'label': 'Perceptions of corruption', 'method': 'update', 'args': [{'y': [df["Perceptions of corruption"]]}]}
]

# Update layout with dropdown menu
fig.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_options,
            'direction': 'down',
            'showactive': True,
        }
    ],
    title='Happiness Dimension vs. Economic Freedom Rank',
    xaxis_title='Economic Freedom Rank',
    yaxis_title='Selected Happiness Dimension',
)

# Show the figure
pio.show(fig)